In [1]:
import pandas as pd
import numpy as np

In [2]:
bureau = pd.read_csv('G:\Kaggle\home credit\data\\bureau.csv')
#len(bureau)

In [15]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [3]:
bureau_bal = pd.read_csv('G:\Kaggle\home credit\data\\bureau_balance.csv')

* Ignoring months_balance col
* Used get dummies for one hot encoding

In [32]:
cat_bu_bal = pd.get_dummies(bureau_bal.select_dtypes('object'))

In [33]:
cat_bu_bal['SK_ID_BUREAU'] = bureau_bal['SK_ID_BUREAU']
cat_bu_bal_agg = cat_bu_bal.groupby('SK_ID_BUREAU').agg(['sum','mean']).reset_index()

In [ ]:
bu_bal_col = ['SK_ID_BUREAU']
for x in cat_bu_bal_agg.columns.levels[0]:
    if(x != 'SK_ID_BUREAU'):
        for y in cat_bu_bal_agg.columns.levels[1][:-1]:
                bu_bal_col.append('bu_bal_%s_%s' % (x,y))

In [35]:
cat_bu_bal_agg.columns=bu_bal_col

In [36]:
cat_bu_bal_agg.head()

,SK_ID_BUREAU,bu_bal_STATUS_0_sum,bu_bal_STATUS_0_mean,bu_bal_STATUS_1_sum,bu_bal_STATUS_1_mean,bu_bal_STATUS_2_sum,bu_bal_STATUS_2_mean,bu_bal_STATUS_3_sum,bu_bal_STATUS_3_mean,bu_bal_STATUS_4_sum,bu_bal_STATUS_4_mean,bu_bal_STATUS_5_sum,bu_bal_STATUS_5_mean,bu_bal_STATUS_C_sum,bu_bal_STATUS_C_mean,bu_bal_STATUS_X_sum,bu_bal_STATUS_X_mean
0,5001709,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402
1,5001710,5,0.060241,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446
2,5001711,3,0.750000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000
3,5001712,10,0.526316,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000
4,5001713,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000


In [39]:
#merging bureau and bureau bal
#this can be grouped directly by SK_ID_CURR and merged with application_train.csv
bu_bal_agg = cat_bu_bal_agg.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], on = 'SK_ID_BUREAU', how = 'left')

* This bureau files contains no of previous loan applications to Home credit for each SK_ID_CURR
* Extracting no of previous applications to Home Credit for each CSK_ID_CURR and merging it with application_train.csv (how = 'left') as a new feature.

In [37]:
prevApp = bureau.groupby('SK_ID_CURR',as_index=False)['SK_ID_BUREAU'].count().rename(columns={'SK_ID_BUREAU': 'prevApp'})

In [14]:
bureau_agg = bureau.drop(columns='SK_ID_BUREAU').groupby('SK_ID_CURR').agg(['count','max','mean','sum']).reset_index()

In [25]:
#tmp = bureau.drop(columns='SK_ID_BUREAU').groupby('SK_ID_CURR').agg(['count','max','mean','sum']).reset_index()

* bureau_agg contains aggregations of all columns.
* Renaming column names now has 2 levels. Merging levels and renaming columns with its appropriate names.

In [ ]:
col = ['SK_ID_CURR']
for x in bureau_agg.columns.levels[0]:
    if(x != 'SK_ID_CURR'):
        for y in bureau_agg.columns.levels[1][:-1]:
                col.append('bureau_%s_%s' % (x,y))
                

* Assigning new column name to aggregated bureau dataframe.


In [20]:
bureau_agg.columns = col

,SK_ID_CURR,fromBureau_DAYS_CREDIT_count,fromBureau_DAYS_CREDIT_max,fromBureau_DAYS_CREDIT_mean,fromBureau_DAYS_CREDIT_sum,fromBureau_CREDIT_DAY_OVERDUE_count,fromBureau_CREDIT_DAY_OVERDUE_max,fromBureau_CREDIT_DAY_OVERDUE_mean,fromBureau_CREDIT_DAY_OVERDUE_sum,fromBureau_DAYS_CREDIT_ENDDATE_count,...,fromBureau_AMT_CREDIT_SUM_OVERDUE_mean,fromBureau_AMT_CREDIT_SUM_OVERDUE_sum,fromBureau_DAYS_CREDIT_UPDATE_count,fromBureau_DAYS_CREDIT_UPDATE_max,fromBureau_DAYS_CREDIT_UPDATE_mean,fromBureau_DAYS_CREDIT_UPDATE_sum,fromBureau_AMT_ANNUITY_count,fromBureau_AMT_ANNUITY_max,fromBureau_AMT_ANNUITY_mean,fromBureau_AMT_ANNUITY_sum
0,100001,7,-49,-735.000000,-5145,7,0,0.0,0,7,...,0.0,0.0,7,-6,-93.142857,-652,7,10822.5,3545.357143,24817.5
1,100002,8,-103,-874.000000,-6992,8,0,0.0,0,6,...,0.0,0.0,8,-7,-499.875000,-3999,7,0.0,0.000000,0.0
2,100003,4,-606,-1400.750000,-5603,4,0,0.0,0,4,...,0.0,0.0,4,-43,-816.000000,-3264,0,NaN,NaN,0.0
3,100004,2,-408,-867.000000,-1734,2,0,0.0,0,2,...,0.0,0.0,2,-382,-532.000000,-1064,0,NaN,NaN,0.0
4,100005,3,-62,-190.666667,-572,3,0,0.0,0,3,...,0.0,0.0,3,-11,-54.333333,-163,3,4261.5,1420.500000,4261.5


In [42]:
# list of categorical variables in bureau.csv for each previous application made to home credit.
(bureau.select_dtypes('object')).head()

,CREDIT_ACTIVE,CREDIT_CURRENCY,CREDIT_TYPE
0,Closed,currency 1,Consumer credit
1,Active,currency 1,Credit card
2,Active,currency 1,Consumer credit
3,Active,currency 1,Credit card
4,Active,currency 1,Consumer credit


In [ ]:
# using get dummies for categorical features
cat= pd.get_dummies(bureau.select_dtypes('object'))
cat['SK_ID_CURR'] = bureau['SK_ID_CURR']


In [64]:
cat.head()

,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,...,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,SK_ID_CURR
0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354
2,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354


* Using sum and mean aggregations for categorical features.
* sum is used to get no of instances and mean to get the normalised value.

In [51]:
bureau_cat = cat.groupby('SK_ID_CURR').agg(['sum','mean']).reset_index()

In [52]:
bureau_cat.head()

SK_ID_CURR CREDIT_ACTIVE_Active           CREDIT_ACTIVE_Bad debt       \
                              sum      mean                    sum mean   
0     100001                    3  0.428571                      0  0.0   
1     100002                    2  0.250000                      0  0.0   
2     100003                    1  0.250000                      0  0.0   
3     100004                    0  0.000000                      0  0.0   
4     100005                    2  0.666667                      0  0.0   

  CREDIT_ACTIVE_Closed           CREDIT_ACTIVE_Sold       \
                   sum      mean                sum mean   
0                    4  0.571429                  0  0.0   
1                    6  0.750000                  0  0.0   
2                    3  0.750000                  0  0.0   
3                    2  1.000000                  0  0.0   
4                    1  0.333333                  0  0.0   

  CREDIT_CURRENCY_currency 1 ...  CREDIT_TYPE_Microloan       \
                         sum ...                    sum mean   
0                          7 ...                      0  0.0   
1                          8 ...                      0  0.0   
2                          4 ...                      0  0.0   
3                          2 ...                      0  0.0   
4                          3 ...                      0  0.0   

  CREDIT_TYPE_Mobile operator loan      CREDIT_TYPE_Mortgage       \
                               sum mean                  sum mean   
0                                0  0.0                    0  0.0   
1                                0  0.0                    0  0.0   
2                                0  0.0                    0  0.0   
3                                0  0.0                    0  0.0   
4                                0  0.0                    0  0.0   

  CREDIT_TYPE_Real estate loan      CREDIT_TYPE_Unknown type of loan       
                           sum mean                              sum mean  
0                            0  0.0                                0  0.0  
1                            0  0.0                                0  0.0  
2                            0  0.0                                0  0.0  
3                            0  0.0                                0  0.0  
4                            0  0.0                                0  0.0  

[5 rows x 47 columns]

In [61]:
cat_col=['SK_ID_CURR']
for x in bureau_cat.columns.levels[0]:
    if(x != 'SK_ID_CURR'):
        for y in bureau_cat.columns.levels[1][:-1]:
                cat_col.append('bureau_%s_%s' % (x,y))        

In [62]:
bureau_cat.columns = cat_col

In [63]:
bureau_cat.head()

,SK_ID_CURR,fromBureau_CREDIT_ACTIVE_Active_sum,fromBureau_CREDIT_ACTIVE_Active_mean,fromBureau_CREDIT_ACTIVE_Bad debt_sum,fromBureau_CREDIT_ACTIVE_Bad debt_mean,fromBureau_CREDIT_ACTIVE_Closed_sum,fromBureau_CREDIT_ACTIVE_Closed_mean,fromBureau_CREDIT_ACTIVE_Sold_sum,fromBureau_CREDIT_ACTIVE_Sold_mean,fromBureau_CREDIT_CURRENCY_currency 1_sum,...,fromBureau_CREDIT_TYPE_Microloan_sum,fromBureau_CREDIT_TYPE_Microloan_mean,fromBureau_CREDIT_TYPE_Mobile operator loan_sum,fromBureau_CREDIT_TYPE_Mobile operator loan_mean,fromBureau_CREDIT_TYPE_Mortgage_sum,fromBureau_CREDIT_TYPE_Mortgage_mean,fromBureau_CREDIT_TYPE_Real estate loan_sum,fromBureau_CREDIT_TYPE_Real estate loan_mean,fromBureau_CREDIT_TYPE_Unknown type of loan_sum,fromBureau_CREDIT_TYPE_Unknown type of loan_mean
0,100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


* Now we have 3 dataframes one for continuous features and the other for categorical and one just with no of previous applications.
* Final dataframes are
    * bu_bal_agg - contains agg data from bureau_balance.csv
    * bureau_cat - contains categorical aggregated data from bureau.csv
    * bureau_agg - contains continuous aggregated data from bureau.csv

* All the above dataframes have SK_ID_CURR. These dataframes can be merged with application_train.csv on = SK_ID_CURR when needed.